In [1]:
import tensorflow as tf
import os
import numpy as np


In [2]:
home = os.environ['HOME']
base_dir = os.path.join(home, 'YogaModel/dataset')
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

In [3]:
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
train_data = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=20
)


test_data_gen = ImageDataGenerator(
    rescale = 1.0/255
)

test_data = test_data_gen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=5
)

Found 4750 images belonging to 107 classes.
Found 1241 images belonging to 107 classes.


In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(107, activation='softmax')
])

2021-10-18 21:22:42.976898: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-18 21:22:42.982921: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [6]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, epochs=100, validation_data = test_data, verbose = 1)
model.save("YogaModel.h5")

2021-10-18 21:23:00.178310: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
 95/238 [==========>...................] - ETA: 1:11 - loss: 4.6821 - accuracy: 0.0095

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:974: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


238/238 [==============================] - 137s 573ms/step - loss: 4.6747 - accuracy: 0.0095 - val_loss: 4.6435 - val_accuracy: 0.0137
Epoch 2/100
238/238 [==============================] - 134s 564ms/step - loss: 4.6326 - accuracy: 0.0187 - val_loss: 4.5049 - val_accuracy: 0.0266
Epoch 3/100
238/238 [==============================] - 134s 561ms/step - loss: 4.4960 - accuracy: 0.0345 - val_loss: 4.2472 - val_accuracy: 0.0661
Epoch 4/100
238/238 [==============================] - 131s 552ms/step - loss: 4.3412 - accuracy: 0.0425 - val_loss: 3.9609 - val_accuracy: 0.1185
Epoch 5/100
238/238 [==============================] - 133s 557ms/step - loss: 4.1801 - accuracy: 0.0665 - val_loss: 3.6341 - val_accuracy: 0.1604
Epoch 6/100
238/238 [==============================] - 135s 567ms/step - loss: 4.0325 - accuracy: 0.0872 - val_loss: 3.4964 - val_accuracy: 0.1612
Epoch 7/100
238/238 [==============================] - 134s 562ms/step - loss: 3.6981 - accuracy: 0.1335 - val_loss: 3.1828 - val_

In [ ]:
!ls -ltr